# Rubik's Cube 3x3x3 SAT Solver trial - CR - Gaetan Jacquemin & Ilann AMIAUD--PLACHY

## systeme d'exploitation

In [10]:
# uncomment according to your OS

solveur_path = "./gophersat_linux64" #linux
# solveur_path = "./gophersat_win64.exe" #windows
# solveur_path = "./gophersat_darwin_amd64" #mac
# solveur_path = "./gophersat_darwin_arm64" #mac



## Modélisation: 

On reprend la même modélisation que le rubikscube 2x2x2 en l'adaptant pour un cube 3x3x3.

Il faut aussi naturellement rajouter plus de cube 3x3x3.

Ce notebook est moins détaillé que le précédent, car la méthode reste la même, pour plus de détails voir le notebook précédent.

### Modélisation du cube : 

![Rubik's Cube faces](./images/rubikscube3_1.drawio.png)
![Rubik's Cube facettes](./images/rubikscube3_2.drawio.png)


### Modélisation des actions :

Par rapport au cube 2x2 il faut pouvoir tourner toutes les faces cette fois, ainsi que les 3 lignes médianes. Et chacune dans tous les mouvements "+90", "-90" et "180".

On utilise donc:

- 3 variables pour définir la quantité de rotation (0:"+90", 1:"-90", 2:"180")

- 9 variables pour définir la face à tourner

### Modélisation de la résolution du cube : 

Semblable au cube 2x2, on définit les faces du cube, les actions possibles, et les contraintes de résolution pour un certain nombre de mouvements possible, et on incrémente ce nombre si nécessaire jusqu'à trouver une solution.

### Cube utilisé pour le cube 3x3 :
<img src="./images/Cube3x3.jpeg" width="500">



## Implémentation:


### Les variables propositionnelles:

In [11]:
# implémentation de la modélisation du probmème

import numpy as np 

t_max = 19 # nombre de mouvements max dans la littérature


num_to_actions = ["90", "-90", "180"]
num_to_faces = ["U", "D", "F", "B", "L", "R", "M", "E", "S"]

actions_to_num = { action : i for i, action in enumerate(num_to_actions) }
faces_to_num = { face : i for i, face in enumerate(num_to_faces) }
       

def variables(n_steps):
    n_faces = 6
    n_cases = 9
    n_couleurs = 6
    R = np.arange(1, (n_steps+1)*n_faces*n_cases*n_couleurs + 1).reshape((n_steps+1), n_faces, n_cases, n_couleurs)

    # les actions (3 mouvements et 3 faces)

    
    n_mouvements = len(num_to_actions)
    n_faces_action = len(num_to_faces)
    # print("n_mouvements:", n_mouvements)
    # print("n_faces_action:", n_faces_action)
    A = np.arange(1, n_steps*n_mouvements  + 1).reshape(n_steps, n_mouvements)
    A += np.max(R)
    F = np.arange(1, n_steps*n_faces_action  + 1).reshape(n_steps, n_faces_action)
    F += np.max(A)
    
    return R, A, F
    
R, A, F = variables(t_max)

print("on s'assure que 2 variables n'ont pas le même nombre: ")
print("R :",R.min(),"->", R.max(), R.shape)
print("A :",A.min(),"->", A.max(), A.shape)
print("F :",F.min(),"->", F.max(), F.shape)
print("on a donc", F.max(), "variables propositionnelles pour notre modélisation.")



on s'assure que 2 variables n'ont pas le même nombre: 
R : 1 -> 6480 (20, 6, 9, 6)
A : 6481 -> 6537 (19, 3)
F : 6538 -> 6708 (19, 9)
on a donc 6708 variables propositionnelles pour notre modélisation.


### Contraintes d'unicité des mouvements:

In [12]:
# CONTRAINTES DE BASE POUR LE CUBE

# en général on utilisera 
# t pour le numéro de l'étape
# f pour la face
# i pour le numéro de la case
# c pour la couleur

def contrainte_unique_couleurs(R):
    
    t_max, n_faces, n_cases, n_couleurs = R.shape

    basic_R_constraints = []

    # chaque case doit être colorié d'au moins une couleur
    for t,f,i in np.ndindex((t_max, n_faces, n_cases)):
        basic_R_constraints.append([int(R[t,f,i,c]) for c in range(6)])

    # chaque case doit être colorié d'au plus une couleur
    for t,f,i in np.ndindex((t_max,n_faces,n_cases)):
        for k in range(n_couleurs):
            for l in range(k+1,n_couleurs):
                basic_R_constraints.append([-int(R[t,f,i,k]), -int(R[t,f,i,l])])
    
    return basic_R_constraints
            
basic_R_clauses_color = contrainte_unique_couleurs(R)
print("nombre de clauses pour l'unicité des couleurs:", len(basic_R_clauses_color))



nombre de clauses pour l'unicité des couleurs: 17280


### Contraintes d'unicité des mouvements:

In [13]:
# CONTRAINTES DE BASE POUR LES MOUVEMENTS

def contraintes_unique_mouvement(M,F):
    t_max, n_mouvements = M.shape
    t_max, n_faces_actions = F.shape

    basic_moves_constraints = []

    # il doit y avoir au moins un mouvement sur une face à chaque étape,
    for t in range(t_max):
        basic_moves_constraints.append([int(M[t,k]) for k in range(n_mouvements)])
        basic_moves_constraints.append([int(F[t,k]) for k in range(n_faces_actions)])
        
        
    # il doit y avoir au plus un mouvement sur une face à chaque étape
    for t in range(t_max):
        for k in range(n_mouvements):
            for l in range(k+1,n_mouvements):
                basic_moves_constraints.append([-int(M[t,k]), -int(M[t,l])])
        for k in range(n_faces_actions):
            for l in range(k+1,n_faces_actions):
                basic_moves_constraints.append([-int(F[t,k]), -int(F[t,l])])
    
    return basic_moves_constraints
            
basic_R_clauses_move = contraintes_unique_mouvement(A,F)
print("nombre de contraintes pour l'unicité des mouvements:", len(basic_R_clauses_move))

nombre de contraintes pour l'unicité des mouvements: 779


### Contraintes de résolution du cube:

In [14]:
# Contraite de résolution du problème

def equal_to(a,b):
    return [[-a,b],[a,-b]]

def implies(a,b):
    return [[-a,b]]

def contraintes_de_resolution(R):
    t_max, n_faces, n_cases, n_couleurs = R.shape
    resolution_clauses = []
    for f in range(n_faces):
        for c in range(n_cases-1):
            for k in range(n_couleurs):
                resolution_clauses += implies(R[t_max-1,f,c,k], R[t_max-1,f,n_cases-1,k])
    return resolution_clauses

resolution_constraints = contraintes_de_resolution(R)
print("nombre de contraintes de résolution:", len(resolution_constraints))

nombre de contraintes de résolution: 288


### Contraintes de mouvement

La logique est la même que le cube 2x2 avec plus de faces et de mouvements possibles. Le code a été réorganisé pour faciliter l'implémentation des contraintes.

In [15]:
# rotation horaire face 1

# on cherche à traduire des implications découlant des actions choisies


# exemple sens horaire : 
# F(t,1) \and M(t,1) => R(t,1,1,:) = R(t+1,1,2,:)
# on peut déjà le transformer en une implication comme expliqué plus haut:
# F(t,1) \and M(t,1) => ( R(t,1,1,:) => R(t+1,1,2,:) ) 
# on peut la transformer en forme normale conjonctive:
# ( -F(t,1) \or -M(t,1) \or -R(t,1,1,:) \or R(t+1,1,2,:) )


# les mouvements sont numérotés comme suis:
adjacences = {
    0 : ((4, 2), (4, 1), (4, 0), (3,2), (3, 1), (3, 0), (2, 2), (2, 1), (2, 0), (1, 2), (1, 1), (1, 0)), # Rotate Up
    1 : ((0, 6), (0, 7), (0, 8), (2, 8), (2, 5), (2, 2), (5, 2), (5, 1), (5, 0), (4, 0), (4, 3), (4, 6)), # Rotate Front
    2 : ((0, 0), (0, 3), (0, 6), (3, 8), (3, 5), (3, 2), (5, 0), (5, 3), (5, 6), (1, 0), (1, 3), (1, 6)), # Rotate Left
    3 : ((0, 2), (0, 1), (0, 0), (4, 8), (4, 5), (4, 2), (5, 6), (5, 7), (5, 8), (2, 0), (2, 3), (2, 6)), # Rotate Back
    4 : ((0, 8), (0, 5), (0, 2), (1, 8), (1, 5), (1, 2), (5, 8), (5, 5), (5, 2), (3, 0), (3, 3), (3, 6)), # Rotate Right
    5 : ((1, 6), (1, 7), (1, 8), (2, 6), (2, 7), (2, 8), (3, 6), (3, 7), (3, 8), (4, 6), (4, 7), (4, 8)), # Rotate Down
    6 : ((4, 5), (4, 4), (4, 3), (3, 5), (3, 4), (3, 3), (2, 5), (2, 4), (2, 3), (1, 5), (1, 4), (1, 3)), # Rotate Middle from Upper view
    7 : ((0, 3), (0, 4), (0, 5), (2, 7), (2, 4), (2, 1), (5, 5), (5, 4), (5, 3), (4, 1), (4, 4), (4, 7)), # Rotate Middle from Front view
    8 :  ((0, 1), (0, 4), (0, 7), (3, 7), (3, 4), (3, 1), (5, 1), (5, 4), (5, 7), (1, 1), (1, 4), (1, 7)), # Rotate Middle from Lefty view
}


increment_adjacences = {
    "90" : lambda i : (i-3) % 12,
    "-90" : lambda i : (i+3) % 12,
    "180" : lambda i : (i+6) % 12
}

increment_faces = {
    "90" : lambda i : (i-2) % 8,
    "-90" : lambda i : (i+2) % 8,
    "180" : lambda i : (i+4) % 8
}


def rotate_face_logic(how="90", action_num=0, step=1, R=None, M=None, F=None):
    
    assert action_num < 6, "action_num doit être inférieur à 6"
    assert how in ["90", "-90", "180"], "how doit être dans ['90', '-90', '180']"
    C = [] #clauses
    
    # face principale:
    ordre = [0,3,6,7,8,5,2,1]
    for i in range(8):
        f1, c1 = action_num, ordre[i]
        f2, c2 = action_num, ordre[increment_faces[how](i)]
        for k in range(6):
            C.append([-F[step,action_num], -M[step,actions_to_num[how]], -R[step,f1,c1,k], R[step+1,f2,c2,k]])
    C.append([-F[step,action_num], -M[step,actions_to_num[how]], -R[step,action_num,4,0], R[step+1,action_num,4,0]])
    
    
    # facettes adjacentes:
    ordre = adjacences[action_num]
    for i in range(len(ordre)):
        f1, c1 = ordre[i]
        f2, c2 = ordre[increment_adjacences[how](i)]
        for k in range(6):
            C.append( [ -F[step,action_num], -M[step,actions_to_num[how]], -R[step,f1,c1,k], R[step+1,f2,c2,k] ]) 
            
    # other faces:
    for face in range(6):
        if face != action_num:
            for c in range(9):
                if (face,c) not in adjacences[action_num]:
                    for k in range(6):
                        C.append([-F[step,action_num],-M[step,actions_to_num[how]], -R[step,face,c,k], R[step+1,face,c,k]])
    return C




def rotate_middle(how="90", action_num=6, step=1, R=None, M=None, F=None):
    
    assert action_num >= 6, "action_num doit être supérieur ou égal à 6"
    assert how in ["90", "-90", "180"], "how doit être dans ['90', '-90', '180']"
    
    C = [] #clauses
    
    # facettes adjacentes:
    ordre = adjacences[action_num]
    for i in range(len(ordre)):
        f1, c1 = ordre[i]
        f2, c2 = ordre[increment_faces[how](i)]
        for k in range(6):
            C.append( [ -F[step,action_num], -M[step,actions_to_num[how]], -R[step,f1,c1,k], R[step+1,f2,c2,k] ]) 
            
    # other faces:
    for face in range(6):
        for c in range(9):
            if (face,c) not in adjacences[action_num]:
                for k in range(6):
                    C.append([-F[step,action_num],-M[step,actions_to_num[how]], -R[step,face,c,k], R[step+1,face,c,k]])
                    
    return C



def contraintes_de_mouvements(R,M,F):
    t_max, n_faces, n_cases, n_couleurs = R.shape
    n_steps, n_actions = F.shape
    
    mouvement_clauses = []

    for t in range(t_max-1):
        for action_num in range(n_actions):
            for how in ["90", "-90", "180"]:
                if action_num < 6:
                    mouvement_clauses += rotate_face_logic(how=how,action_num=action_num,step=t,R=R,M=M,F=F)
                else:
                    mouvement_clauses += rotate_middle(how=how,action_num=action_num,step=t,R=R,M=M,F=F)
                                
    return mouvement_clauses

mouvement_clauses = contraintes_de_mouvements(R,A,F)
print("nombre de clauses:", len(mouvement_clauses))


nombre de clauses: 164502


### Contraintes de départ

De même, on définit les contraintes de départ pour un cube 3x3x3, et on a réorganisé le code pour que cela soit plus lisible et simple dû au nombre de mouvements plus importants. Pour le mélange du cube, on n'encodera qu'un mouvement de rotation à 90 degrés pour chaque face, ce qui suffit pour mélanger le cube et permettre de créer tous les cubes résolvables.

In [16]:

import random


COLORS = {
    0: "🟥",  # Rouge (Up)
    1: "🟩",  # Vert (Front)
    2: "🟦",  # Bleu (Left)
    3: "🟧",  # Orange (Back)
    4: "🟨",  # Jaune (Down)
    5: "⬜"   # Blanc (Right)
}


text_2_face = {
    "Up": 0,
    "Front": 1,
    "Left": 2,
    "Back": 3,
    "Right": 4,
    "Down": 5,
}

# Création du cube 2x2x2 correctement formaté (6 faces, 4)
def create_cube():
    return np.array([[i, i, i, i, i, i, i, i, i] for i in range(6)])

# Affichage amélioré du cube en format "déplié" pour un cube 3x3x3
def print_cube(cube):
    print(f"          {COLORS[int(cube[0][0])]} {COLORS[int(cube[0][1])]} {COLORS[int(cube[0][2])]}")
    print(f"          {COLORS[int(cube[0][3])]} {COLORS[int(cube[0][4])]} {COLORS[int(cube[0][5])]}")
    print(f"          {COLORS[int(cube[0][6])]} {COLORS[int(cube[0][7])]} {COLORS[int(cube[0][8])]}\n")

    for i in range(3):
        print(f"{COLORS[int(cube[2][i*3])]} {COLORS[int(cube[2][i*3+1])]} {COLORS[int(cube[2][i*3+2])]}  "
            f"{COLORS[int(cube[1][i*3])]} {COLORS[int(cube[1][i*3+1])]} {COLORS[int(cube[1][i*3+2])]}  "
            f"{COLORS[int(cube[4][i*3])]} {COLORS[int(cube[4][i*3+1])]} {COLORS[int(cube[4][i*3+2])]}  "
            f"{COLORS[int(cube[3][i*3])]} {COLORS[int(cube[3][i*3+1])]} {COLORS[int(cube[3][i*3+2])]}")
    print()

    print(f"          {COLORS[int(cube[5][0])]} {COLORS[int(cube[5][1])]} {COLORS[int(cube[5][2])]}")
    print(f"          {COLORS[int(cube[5][3])]} {COLORS[int(cube[5][4])]} {COLORS[int(cube[5][5])]}")
    print(f"          {COLORS[int(cube[5][6])]} {COLORS[int(cube[5][7])]} {COLORS[int(cube[5][8])]}\n")

def move_edge(cube, from_face, from_indices, to_face, to_indices, temp):
    """Déplace les éléments d'un bord d'une face à une autre."""
    for i in range(3):
        cube[to_face][to_indices[i]] = temp[from_face][from_indices[i]]
    
def rotate_principal_face( cube, face ):
    if face >= 3:
        cube[face] = [ cube[face][2], cube[face][5], cube[face][8], cube[face][1], cube[face][4], cube[face][7], cube[face][0], cube[face][3], cube[face][6] ]
    else:  
        cube[face] = [ cube[face][6], cube[face][3], cube[face][0], cube[face][7], cube[face][4], cube[face][1], cube[face][8], cube[face][5], cube[face][2] ]

def rotate_face(cube, face):
    """
    Effectue une rotation de 90° dans le sens horaire sur la face spécifiée et met à jour les bords.
    """
    temp = cube.copy()
    
    if face == text_2_face["Front"] or face == text_2_face["Back"]:  # Face avant
        # Mise à jour des bords (Haut → Droite, Droite → Bas, Bas → Gauche, Gauche → Haut)
        line_up = [6, 7, 8] if face == text_2_face["Front"] else [0, 1, 2]
        line_right = [0, 3, 6] if face == text_2_face["Front"] else [2, 5, 8]
        line_down = [2, 1, 0] if face == text_2_face["Front"] else [8, 7, 6]
        line_left = [8, 5, 2] if face == text_2_face["Front"] else [6, 3, 0]

        for from_face, to_face, from_line, to_line in [(text_2_face["Up"], text_2_face["Right"], line_up, line_right),
                                                    (text_2_face["Right"], text_2_face["Down"], line_right, line_down),
                                                    (text_2_face["Down"], text_2_face["Left"], line_down, line_left),
                                                    (text_2_face["Left"], text_2_face["Up"], line_left, line_up)]:
            move_edge(cube, from_face, from_line, to_face, to_line, temp)
        

    if face == text_2_face["Up"] or face == text_2_face["Down"]:  # Face Haut
        line = [0, 1, 2] if face == text_2_face["Up"] else [6, 7, 8]
        for from_face, to_face in [(text_2_face["Front"], text_2_face["Left"]), (text_2_face["Left"], text_2_face["Back"]),
                                (text_2_face["Back"], text_2_face["Right"]), (text_2_face["Right"], text_2_face["Front"])]:
            move_edge(cube, from_face, line, to_face, line, temp)
        
        
    if face == text_2_face["Left"] or face == text_2_face["Right"]: #Face gauche
        line = [0, 3, 6] if face == text_2_face["Left"] else [8, 5, 2]
        back_line = [8, 5, 2] if face == text_2_face["Left"] else [0, 3, 6]
        for from_face, to_face, from_line, to_line in [(text_2_face["Back"],text_2_face["Up"], back_line, line),
                                                    (text_2_face["Down"], text_2_face["Back"],  line, back_line),
                                                    (text_2_face["Front"], text_2_face["Down"],  line, line),
                                                    ( text_2_face["Up"], text_2_face["Front"], line, line)]:
            move_edge(cube, from_face, from_line, to_face, to_line, temp)
        
    
    
    # Rotation autour de la face
    
    rotate_principal_face(cube, face)
    
    return cube

# Exécution
cube = create_cube()
print("Cube initial :")
print_cube(cube)


n = 6 # Nombre de rotations aléatoires
for i in range(n):
    cube = rotate_face(cube, random.randint(0,5))  # Rotation de la face FRONT
    #print(f"Après {i+1} rotations :")
    #print_cube(cube)

    

print(f"Après {n} rotations :")
print_cube(cube)

def generate_cube(n):
    cube = create_cube()
    for i in range(n):
        cube = rotate_face(cube, random.randint(0,5))
    return cube



# Ajouter les contraintes à notre problème

def contraintes_initiales(R, cube):
    t_max, n_faces, n_cases, n_couleurs = R.shape
    
    contraintes = []
    
    for f in range(n_faces):
        for i in range(n_cases):
            for c in range(n_couleurs):
                if cube[f][i] == c:
                    contraintes.append([R[0,f,i,c]])
    return contraintes

initial_constraints = contraintes_initiales(R, cube)
print( "nombre de contraintes initiales:", len(initial_constraints) )



Cube initial :
          🟥 🟥 🟥
          🟥 🟥 🟥
          🟥 🟥 🟥

🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧
🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧
🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧

          ⬜ ⬜ ⬜
          ⬜ ⬜ ⬜
          ⬜ ⬜ ⬜

Après 6 rotations :
          🟩 🟥 ⬜
          🟦 🟥 🟧
          🟦 🟥 ⬜

🟨 🟩 🟩  🟥 🟨 🟦  🟧 🟥 🟧  🟨 🟦 ⬜
⬜ 🟦 ⬜  🟨 🟩 🟦  🟧 🟨 🟧  🟨 🟧 🟧
🟥 🟩 🟦  🟧 🟥 🟥  🟨 🟦 🟦  🟩 🟩 🟨

          🟥 🟩 🟩
          🟨 ⬜ ⬜
          🟧 ⬜ ⬜

nombre de contraintes initiales: 54


## Résolution

In [17]:
from tqdm import tqdm
import subprocess


def generate_cnf_file(n_steps, cube):
    
    R, M, F = variables(n_steps)
    clauses = []
    clauses += contrainte_unique_couleurs(R)
    clauses += contraintes_unique_mouvement(M,F)
    clauses += contraintes_de_resolution(R)
    clauses += contraintes_de_mouvements(R,M,F)
    clauses += contraintes_initiales(R, cube)
    
    # print(f"pour {n_steps} steps:")
    # print("nombre de variables:", F.max())
    # print("nombre de clauses:", len(clauses))
    
    with open("./cnf/cube_"+str(n_steps)+".cnf", "w") as f:
        f.write("c Generated by rubik_cube_cnf.py\n")
        f.write(f"c for the resolution of the cune in {n_steps} steps\n")
        f.write("c\n")
        f.write(f"p cnf {F.max()} {len(clauses)}\n")
        for clause in clauses:
            for variable in clause:
                f.write(str(variable) + " ")
            f.write("0\n")
    
    # print("\n")
    
#test
#generate_cnf_file(2, cube)
    
    
# Fonction pour exécuter les commandes et vérifier la sortie
def check_sat_solution(cube):
    for n in tqdm(range(1, 20)):  # n entre 1 et 19
        
        generate_cnf_file(n, cube)
        
        #command = f"./gophersat_linux64 cube_{n}.cnf"  # Commande à exécuter (à changer en fonction de votre système d'exploitation)
        command = solveur_path + f" ./cnf/cube_{n}.cnf"
        
        result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Vérification si la sortie contient "SATISFIABLE"
        output = result.stdout.decode('utf-8')
        if "UNSATISFIABLE" not in output:
            print(f"SATISFIABLE trouvé pour cube_{n}.cnf")

            # Recherche de la décomposition dans la sortie
            lines = output.splitlines()
            final_line = None
            for line in lines:
                if line.startswith("v"):
                    final_line = line
                    final_line = list(map(lambda x: 0 if int(x)<0 else 1, final_line.strip(" 0/n").replace("v ", "").split(" ")))
                    # print(final_line)
                    
            return n, final_line
        else:
            if "Exec format error" in output:
                print("Erreur d'exécution, vérifiez le chemin du solveur en haut de ce jupyter")
            # print(f"UNSATISFIABLE pour cube_{n}.cnf")
            
def resolve(cube):

    print("Résolution du cube...")
    n, line = check_sat_solution(cube)

    # Analyse des résultats
    Rmax = (n+1)*6*9*6
    Amax = Rmax + n*3
    Fmax = Rmax + n*9 
    
    num_2_face = {
        0: "Up",
        1: "Front",
        2: "Left",
        3: "Back",
        4: "Right",
        5: "Down"
    }  
    
    move_to_num = { 
        0: {"90": 1,
        "-90": 3,
        "180": 2,
        },
        1: {"90": 3,
        "-90": 1,
        "180": 2,
        },
    }
    
    
    # R = np.array(line[0:Rmax]).reshape((n+1,6,9,6))
    # print(R)
    A = np.array(line[Rmax:Amax]).reshape( (n,3) )

    F = np.array(line[Amax:]).reshape( (n,9) )


    print("\nSatisfiable pour", n, "étapes")

    for t in range(n):
        print("step", t,end=": ")
        for i in range(9):
            if F[t,i] != 0:
                face = i
                print(num_2_face[i], end=" ")
        for i in range(3):
            if A[t,i] != 0:
                action = num_to_actions[i]
                print(action)
        for i in range( move_to_num[face // 3][action] ):
            cube = rotate_face(cube, int(face))
        print_cube(cube)
        

In [19]:
#### Génération du cube ####

n = 7 # Nombre de rotations aléatoires
# Attention ! Le temps de résolution peut être très long pour n > 7.
cube = generate_cube(n)
print("Cube à résoudre :")
print_cube(cube)

#### Résolution du cube ####

resolve(cube)


Cube à résoudre :
          ⬜ 🟩 🟥
          ⬜ 🟥 🟥
          🟧 🟧 🟦

🟨 🟩 🟥  🟨 🟨 🟧  ⬜ 🟧 🟧  🟦 🟨 🟧
⬜ 🟦 🟦  🟥 🟩 🟥  🟨 🟨 🟧  🟦 🟧 🟦
🟩 ⬜ 🟨  🟩 🟩 ⬜  🟦 🟦 ⬜  🟩 ⬜ 🟥

          🟥 🟥 🟩
          🟨 ⬜ 🟩
          🟦 🟧 🟨

Résolution du cube...


 32%|███▏      | 6/19 [06:34<14:15, 65.79s/it]

SATISFIABLE trouvé pour cube_7.cnf

Satisfiable pour 7 étapes
step 0: Back 90
          🟧 🟧 ⬜
          ⬜ 🟥 🟥
          🟧 🟧 🟦

🟥 🟩 🟥  🟨 🟨 🟧  ⬜ 🟧 🟨  🟩 🟦 🟦
🟩 🟦 🟦  🟥 🟩 🟥  🟨 🟨 🟧  ⬜ 🟧 🟨
⬜ ⬜ 🟨  🟩 🟩 ⬜  🟦 🟦 🟦  🟥 🟦 🟧

          🟥 🟥 🟩
          🟨 ⬜ 🟩
          🟨 ⬜ 🟩

step 1: Up -90
          ⬜ 🟥 🟦
          🟧 🟥 🟧
          🟧 ⬜ 🟧

🟩 🟦 🟦  🟥 🟩 🟥  🟨 🟨 🟧  ⬜ 🟧 🟨
🟩 🟦 🟦  🟥 🟩 🟥  🟨 🟨 🟧  ⬜ 🟧 🟨
⬜ ⬜ 🟨  🟩 🟩 ⬜  🟦 🟦 🟦  🟥 🟦 🟧

          🟥 🟥 🟩
          🟨 ⬜ 🟩
          🟨 ⬜ 🟩

step 2: Left -90
          🟥 🟥 🟦
          🟥 🟥 🟧
          🟩 ⬜ 🟧

🟦 🟦 🟨  🟥 🟩 🟥  🟨 🟨 🟧  ⬜ 🟧 🟧
🟦 🟦 ⬜  🟨 🟩 🟥  🟨 🟨 🟧  ⬜ 🟧 🟧
🟩 🟩 ⬜  🟨 🟩 ⬜  🟦 🟦 🟦  🟥 🟦 ⬜

          🟧 🟥 🟩
          🟨 ⬜ 🟩
          🟨 ⬜ 🟩

step 3: Right 90
          🟥 🟥 🟥
          🟥 🟥 🟥
          🟩 ⬜ ⬜

🟦 🟦 🟨  🟥 🟩 🟩  🟦 🟨 🟨  🟧 🟧 🟧
🟦 🟦 ⬜  🟨 🟩 🟩  🟦 🟨 🟨  🟧 🟧 🟧
🟩 🟩 ⬜  🟨 🟩 🟩  🟦 🟧 🟧  🟦 🟦 ⬜

          🟧 🟥 🟥
          🟨 ⬜ ⬜
          🟨 ⬜ ⬜

step 4: Front -90
          🟥 🟥 🟥
          🟥 🟥 🟥
          🟦 🟦 🟦

🟦 🟦 ⬜  🟩 🟩 🟩  🟥 🟨 🟨  🟧 🟧 🟧
🟦 🟦 ⬜  🟩 🟩 🟩  🟥 🟨 🟨  🟧 🟧 🟧
🟩 🟩 🟩  🟥 🟨 🟨  🟧 🟧 🟧  🟦 🟦 ⬜

    